# MITRE ATLAS + YOLO demo (Jupyter)
This notebook gives a cell-by-cell walkthrough for exploring the ATLAS mapping, planning mitigations, and running detections on maritime imagery.

## Setup
The core mapping utilities use only the standard library. If you want to run real YOLOv5 detections, install the optional extras in a terminal before executing the relevant cell:

```bash
pip install -e .[yolo]
```

If you already have a `torch` installation (CPU or GPU), the analyzer will reuse it.

In [ ]:
from atlas_mapping import MILLIAMPERE2_PROFILE, build_mitigation_plan
from atlas_mapping.planner import plan_summaries

plans = build_mitigation_plan(
    nodes=MILLIAMPERE2_PROFILE['nodes'],
    threats=MILLIAMPERE2_PROFILE['threats'],
    defences=MILLIAMPERE2_PROFILE['defences'],
)

print(plan_summaries(plans))

## Run a quick dry-run analyzer (no YOLO required)
This helper backend emits deterministic fake detections so you can see how detections, nodes, and mitigations are bound together.

In [ ]:
from dataclasses import dataclass
from typing import Iterable
from atlas_mapping.image_analyzer import Detection, MaritimeImageAnalyzer

@dataclass
class DemoBackend:
    def predict(self, image_path: str) -> Iterable[Detection]:
        # Fixed detections for demonstration
        yield Detection(label='vessel', confidence=0.92, bbox=[120.0, 80.0, 260.0, 220.0])
        yield Detection(label='dock', confidence=0.81, bbox=[20.0, 140.0, 180.0, 260.0])

demo_analyzer = MaritimeImageAnalyzer(backend=DemoBackend())
demo_result = demo_analyzer.analyze_image('demo_maritime_frame.jpg')
print(demo_result.summary())

## (Optional) Run YOLOv5 on your maritime image
Uncomment and edit the `image_path` to point at a real image. This cell will use the Ultralytics YOLOv5 model if the optional dependencies are installed.

In [ ]:
# Uncomment the next line and set it to a real image before running this cell.
# image_path = '/absolute/path/to/your_maritime_image.jpg'

# from atlas_mapping.image_analyzer import MaritimeImageAnalyzer, UltralyticsYolov5Backend

# try:
#     backend = UltralyticsYolov5Backend(model_name='yolov5s', device='cpu')
#     analyzer = MaritimeImageAnalyzer(backend=backend)
#     if 'image_path' not in locals():
#         raise FileNotFoundError('Set `image_path` to a real file path before running YOLO inference.')
#     yolo_result = analyzer.analyze_image(image_path)
#     print(yolo_result.summary())
# except Exception as exc:
#     print(f'Skipping YOLO run: {exc}')
